# scratch work

In [1]:
import jax

jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp
import rebound
import reboundx

sim = rebound.Simulation()
sim.add(m=1.0, x=-1.0, y=1.01, z=-0.05, vx=0.01, vy=0.02, vz=0.4)
sim.add(m=1.3, x=1.0, y=-2.01, z=0.04, vx=-0.02, vy=-0.05, vz=-0.6)
sim.add(m=0.01, x=10.0, y=-2.01, z=0.04, vx=-0.02, vy=-0.05, vz=-0.6)
sim.add(m=0.01, x=10.0, y=-2.01, z=0.4, vx=-0.02, vy=-0.05, vz=0.6)
sim.integrate(1e-300)
a0 = jnp.array([[p.ax, p.ay, p.az] for p in sim.particles])
sim = rebound.Simulation()
sim.add(m=1.0, x=-1.0, y=1.01, z=-0.05, vx=0.01, vy=0.02, vz=0.4)
sim.add(m=1.3, x=1.0, y=-2.01, z=0.04, vx=-0.02, vy=-0.05, vz=-0.6)
sim.add(m=0.01, x=10.0, y=-2.01, z=0.04, vx=-0.02, vy=-0.05, vz=-0.6)
sim.add(m=0.01, x=10.0, y=-2.01, z=0.4, vx=-0.02, vy=-0.05, vz=0.6)
rebx = reboundx.Extras(sim)
gr = rebx.load_force("gr_full")
gr.params["c"] = 10
gr.params["max_iterations"] = 10
rebx.add_force(gr)
sim.integrate(1e-300)
a1 = sim.particles[0].ax
print(sim.particles[1].ax)
print(sim.particles[1].ay)
print(sim.particles[1].az)
a1 = jnp.array([[p.ax, p.ay, p.az] for p in sim.particles])
reb_res = jnp.array([sim.particles[1].ax, sim.particles[1].ay, sim.particles[1].az])

-0.04134904004818635
0.06280271508706053
-0.0019105835434115458


In [2]:
import jax

jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp

from copy import deepcopy
import math
import numpy as np
import sys
import rebound


def rebx_calculate_gr_full(
    particles, C2, G, max_iterations=10, gravity_ignore_10=False
):
    xs = np.array([[p.x] for p in particles])
    ys = np.array([[p.y] for p in particles])
    zs = np.array([[p.z] for p in particles])
    vxs = np.array([[p.vx] for p in particles])
    vys = np.array([[p.vy] for p in particles])
    vzs = np.array([[p.vz] for p in particles])
    gms = np.array([[p.m] for p in particles])

    x = jnp.array([[p.x, p.y, p.z] for p in sim.particles])
    v = jnp.array([[p.vx, p.vy, p.vz] for p in sim.particles])
    gms = jnp.array([p.m for p in sim.particles])

    N = x.shape[0]
    # C2 = SPEED_OF_LIGHT**2
    C2 = 100.0

    # Calculate pairwise differences
    dx = x[:, None, :] - x[None, :, :]  # (N,N,3)
    dv = v[:, None, :] - v[None, :, :]  # (N,N,3)
    r2 = jnp.sum(dx * dx, axis=-1)  # (N,N)
    r = jnp.sqrt(r2)  # (N,N)
    r3 = r2 * r  # (N,N)

    # Mask for i!=j calculations
    mask = ~jnp.eye(N, dtype=bool)  # (N,N)

    # Compute initial Newtonian accelerations
    prefac = 1.0 / r3
    prefac = jnp.where(mask, prefac, 0.0)
    a_newt = -jnp.sum(prefac[:, :, None] * dx * gms[None, :, None], axis=1)  # (N,3)

    # Move to barycentric frame
    x_com = jnp.sum(x * gms[:, None], axis=0) / jnp.sum(gms)
    v_com = jnp.sum(v * gms[:, None], axis=0) / jnp.sum(gms)
    x = x - x_com
    v = v - v_com
    v2 = jnp.sum(v * v, axis=-1)

    # Compute constant acceleration terms
    a_const = np.zeros((N, 3))

    for i in range(N):
        for j in range(N):
            if j == i:
                continue

            # First constant part calculations
            a1 = jnp.sum((4.0 / C2) * gms / r[i], where=mask[i])

            a2 = jnp.sum((1.0 / C2) * gms / r[j], where=mask[j])

            a3 = -v2[i] / C2

            a4 = -2 * v2[j] / C2

            a5 = (4 / C2) * jnp.sum(v[i] * v[j], axis=-1)

            a6_0 = jnp.sum(dx[i, j] * v[j])
            a6 = (3 / (2 * C2)) * a6_0 * a6_0 / r2[i, j]

            a7 = jnp.sum(dx[i, j] * a_newt[j]) / (2 * C2)

            factor1 = a1 + a2 + a3 + a4 + a5 + a6 + a7

            a_const[i] += gms[j] * dx[i, j] * factor1 / r3[i, j, None]

            # Second constant part calculations
            factor2 = jnp.sum(dx[i, j] * (4 * v[i] - 3 * v[j]))

            a_const[i] += (
                gms[j]
                / C2
                * (
                    factor2 * dv[i, j] / r3[i, j, None]
                    + 7 / 2 * a_newt[j] / r[i, j, None]
                )
            )

    print(f"a_const: {a_const[0,0]}")
    ps_b = deepcopy(particles)
    for i, p in enumerate(ps_b):
        p.ax = a_newt[i, 0]
        p.ay = a_newt[i, 1]
        p.az = a_newt[i, 2]
    _q = rebound.Simulation()
    _q.add(ps_b)
    _q.move_to_com()
    ps_b = list(_q.particles)
    # Set initial accelerations to constant terms
    for i in range(N):
        ps_b[i].ax = a_const[i][0]
        ps_b[i].ay = a_const[i][1]
        ps_b[i].az = a_const[i][2]

    # Iterative refinement of accelerations
    for k in range(10):  # Maximum 10 iterations
        # Store old accelerations
        a_old = np.array([[p.ax, p.ay, p.az] for p in ps_b])

        # Add non-constant term
        for i in range(N):
            non_constx, non_consty, non_constz = 0.0, 0.0, 0.0

            for j in range(N):
                if j != i:
                    dxij = ps_b[i].x - ps_b[j].x
                    dyij = ps_b[i].y - ps_b[j].y
                    dzij = ps_b[i].z - ps_b[j].z

                    rij = math.sqrt(dxij * dxij + dyij * dyij + dzij * dzij)
                    rij3 = rij * rij * rij

                    dotproduct = (
                        dxij * ps_b[j].ax + dyij * ps_b[j].ay + dzij * ps_b[j].az
                    )

                    non_constx += (G * particles[j].m * dxij / rij3) * dotproduct / (
                        2 * C2
                    ) + (7 / (2 * C2)) * G * particles[j].m * ps_b[j].ax / rij
                    non_consty += (G * particles[j].m * dyij / rij3) * dotproduct / (
                        2 * C2
                    ) + (7 / (2 * C2)) * G * particles[j].m * ps_b[j].ay / rij
                    non_constz += (G * particles[j].m * dzij / rij3) * dotproduct / (
                        2 * C2
                    ) + (7 / (2 * C2)) * G * particles[j].m * ps_b[j].az / rij

            ps_b[i].ax = a_const[i][0] + non_constx
            ps_b[i].ay = a_const[i][1] + non_consty
            ps_b[i].az = a_const[i][2] + non_constz

        # Check for convergence
        maxdev = 0.0
        for i in range(N):
            dx = (
                abs((ps_b[i].ax - a_old[i][0]) / ps_b[i].ax)
                if abs(ps_b[i].ax) > sys.float_info.epsilon
                else 0
            )
            dy = (
                abs((ps_b[i].ay - a_old[i][1]) / ps_b[i].ay)
                if abs(ps_b[i].ay) > sys.float_info.epsilon
                else 0
            )
            dz = (
                abs((ps_b[i].az - a_old[i][2]) / ps_b[i].az)
                if abs(ps_b[i].az) > sys.float_info.epsilon
                else 0
            )

            maxdev = max(maxdev, dx, dy, dz)

        if maxdev < sys.float_info.epsilon:
            break

        if k == 9:
            print(
                f"Warning: 10 loops in GR calculation did not converge. Fractional Error: {maxdev}"
            )

    # Update original particles with calculated accelerations
    for i in range(N):
        particles[i].ax += ps_b[i].ax
        particles[i].ay += ps_b[i].ay
        particles[i].az += ps_b[i].az

    return particles


sim = rebound.Simulation()
sim.add(m=1.0, x=-1.0, y=1.01, z=-0.05, vx=0.01, vy=0.02, vz=0.4)
sim.add(m=1.3, x=1.0, y=-2.01, z=0.04, vx=-0.02, vy=-0.05, vz=-0.6)
sim.add(m=0.01, x=10.0, y=-2.01, z=0.04, vx=-0.02, vy=-0.05, vz=-0.6)
sim.add(m=0.01, x=10.0, y=-2.01, z=0.4, vx=-0.02, vy=-0.05, vz=0.6)
sim.integrate(1e-300)
particles = list(sim.particles)

p1, p2, p3, p4 = rebx_calculate_gr_full(
    particles, C2=10**2, G=1.0, max_iterations=10, gravity_ignore_10=False
)
py_res = jnp.array([p2.ax, p2.ay, p2.az])

py_res - reb_res

a_const: -0.0006170686149108296


Array([0., 0., 0.], dtype=float64)

In [3]:
def ac(x, v, gms):
    # x is N x 3 array of positions
    # v is N x 3 array of velocities
    # gms is N array of masses

    N = x.shape[0]
    C2 = 100.0

    # Calculate pairwise differences
    dx = x[:, None, :] - x[None, :, :]  # (N,N,3)
    dv = v[:, None, :] - v[None, :, :]  # (N,N,3)
    r2 = jnp.sum(dx * dx, axis=-1)  # (N,N)
    r = jnp.sqrt(r2)  # (N,N)
    r3 = r2 * r  # (N,N)

    # Mask for i!=j calculations
    mask = ~jnp.eye(N, dtype=bool)  # (N,N)

    # Compute initial Newtonian accelerations
    prefac = 1.0 / r3
    prefac = jnp.where(mask, prefac, 0.0)
    a_newt = -jnp.sum(prefac[:, :, None] * dx * gms[None, :, None], axis=1)  # (N,3)

    # Move to barycentric frame
    x_com = jnp.sum(x * gms[:, None], axis=0) / jnp.sum(gms)
    v_com = jnp.sum(v * gms[:, None], axis=0) / jnp.sum(gms)
    x = x - x_com
    v = v - v_com
    v2 = jnp.sum(v * v, axis=-1)

    # Compute constant acceleration terms
    a_const = np.zeros((N, 3))

    for i in range(N):
        for j in range(N):
            if j == i:
                continue

            # First constant part calculations
            a1 = jnp.sum((4.0 / C2) * gms / r[i], where=mask[i])

            a2 = jnp.sum((1.0 / C2) * gms / r[j], where=mask[j])

            a3 = -v2[i] / C2

            a4 = -2 * v2[j] / C2

            a5 = (4 / C2) * jnp.sum(v[i] * v[j], axis=-1)

            a6_0 = jnp.sum(dx[i, j] * v[j])
            a6 = (3 / (2 * C2)) * a6_0 * a6_0 / r2[i, j]

            a7 = jnp.sum(dx[i, j] * a_newt[j]) / (2 * C2)

            factor1 = a1 + a2 + a3 + a4 + a5 + a6 + a7

            a_const[i] += gms[j] * dx[i, j] * factor1 / r3[i, j, None]

            # Second constant part calculations
            factor2 = jnp.sum(dx[i, j] * (4 * v[i] - 3 * v[j]))

            a_const[i] += (
                gms[j]
                / C2
                * (
                    factor2 * dv[i, j] / r3[i, j, None]
                    + 7 / 2 * a_newt[j] / r[i, j, None]
                )
            )

    return a_const


sim = rebound.Simulation()
sim.add(m=1.0, x=-1.0, y=1.01, z=-0.05, vx=0.01, vy=0.02, vz=0.4)
sim.add(m=1.3, x=1.0, y=-2.01, z=0.04, vx=-0.02, vy=-0.05, vz=-0.6)
sim.add(m=0.01, x=10.0, y=-2.01, z=0.04, vx=-0.02, vy=-0.05, vz=-0.6)
sim.add(m=0.01, x=10.0, y=-2.01, z=0.4, vx=-0.02, vy=-0.05, vz=0.6)
sim.integrate(1e-300)
x0 = jnp.array([[p.x, p.y, p.z] for p in sim.particles])
v0 = jnp.array([[p.vx, p.vy, p.vz] for p in sim.particles])
gms = jnp.array([p.m for p in sim.particles])
float(ac(x0, v0, gms)[0, 0])

-0.0006170686149108296

In [54]:
def ac(x, v, gms):
    # x is N x 3 array of positions
    # v is N x 3 array of velocities
    # gms is N array of masses

    N = x.shape[0]
    C2 = 100.0

    # Calculate pairwise differences
    dx = x[:, None, :] - x[None, :, :]  # (N,N,3)
    dv = v[:, None, :] - v[None, :, :]  # (N,N,3)
    r2 = jnp.sum(dx * dx, axis=-1)  # (N,N)
    r = jnp.sqrt(r2)  # (N,N)
    r3 = r2 * r  # (N,N)

    # Mask for i!=j calculations
    mask = ~jnp.eye(N, dtype=bool)  # (N,N)

    # Compute initial Newtonian accelerations
    prefac = 1.0 / r3
    prefac = jnp.where(mask, prefac, 0.0)
    a_newt = -jnp.sum(prefac[:, :, None] * dx * gms[None, :, None], axis=1)  # (N,3)

    # Move to barycentric frame
    x_com = jnp.sum(x * gms[:, None], axis=0) / jnp.sum(gms)
    v_com = jnp.sum(v * gms[:, None], axis=0) / jnp.sum(gms)
    x = x - x_com
    v = v - v_com
    v2 = jnp.sum(v * v, axis=-1)
    vdot = jnp.sum(v[:, None, :] * v[None, :, :], axis=-1)

    # Compute constant acceleration terms
    # first part of the constant term
    a1_arr = jnp.sum((4.0 / C2) * gms / r, axis=1, where=mask)
    a1_arr = jnp.broadcast_to(a1_arr, (N, N)).T

    a2_arr = jnp.sum((1.0 / C2) * gms / r, axis=1, where=mask)
    a2_arr = jnp.broadcast_to(a2_arr, (N, N))

    a3_arr = jnp.broadcast_to(-v2 / C2, (N, N)).T

    a4_arr = -2.0 * jnp.broadcast_to(v2, (N, N)) / C2

    a5_arr = (4.0 / C2) * vdot

    a6_0 = jnp.sum(dx * v[None, :, :], axis=-1)
    a6_arr = (3.0 / (2 * C2)) * (a6_0**2) / r2

    a7_arr = jnp.sum(dx * a_newt[None, :, :], axis=-1) / (2 * C2)

    factor1_arr = a1_arr + a2_arr + a3_arr + a4_arr + a5_arr + a6_arr + a7_arr
    part1_arr = (
        jnp.broadcast_to(gms, (N, N))[:, :, None]
        * dx
        * factor1_arr[:, :, None]
        / r3[:, :, None]
    )

    # second part of the constant term
    factor2_arr = jnp.sum(dx * (4 * v[:, None, :] - 3 * v[None, :, :]), axis=-1)

    part2_arr = (
        jnp.broadcast_to(gms, (N, N))[:, :, None]
        * (
            (
                factor2_arr[:, :, None] * dv / r3[:, :, None]
                + 7 / 2 * a_newt[None, :, :] / r[:, :, None]
            )
        )
        / C2
    )

    a_const_arr = part1_arr + part2_arr

    # a_const = np.zeros((N, 3))
    # for i in range(N):
    #     for j in range(N):
    #         if j == i:
    #             continue

    #         a_const[i] += a_const_arr[i,j]
    a_const = jnp.sum(a_const_arr, axis=1, where=mask[:, :, None])

    return a_const


sim = rebound.Simulation()
sim.add(m=1.0, x=-1.0, y=1.01, z=-0.05, vx=0.01, vy=0.02, vz=0.4)
sim.add(m=1.3, x=1.0, y=-2.01, z=0.04, vx=-0.02, vy=-0.05, vz=-0.6)
sim.add(m=0.01, x=10.0, y=-2.01, z=0.04, vx=-0.02, vy=-0.05, vz=-0.6)
sim.add(m=0.01, x=10.0, y=-2.01, z=0.4, vx=-0.02, vy=-0.05, vz=0.6)
sim.integrate(1e-300)
x0 = jnp.array([[p.x, p.y, p.z] for p in sim.particles])
v0 = jnp.array([[p.vx, p.vy, p.vz] for p in sim.particles])
gms = jnp.array([p.m for p in sim.particles])
float(ac(x0, v0, gms)[0, 0])

-0.0006170686149108297

In [53]:
x = jnp.array([[p.x, p.y, p.z] for p in sim.particles])
v = jnp.array([[p.vx, p.vy, p.vz] for p in sim.particles])
gms = jnp.array([p.m for p in sim.particles])
N = x.shape[0]
C2 = 100.0

# Calculate pairwise differences
dx = x[:, None, :] - x[None, :, :]  # (N,N,3)
dv = v[:, None, :] - v[None, :, :]  # (N,N,3)
r2 = jnp.sum(dx * dx, axis=-1)  # (N,N)
r = jnp.sqrt(r2)  # (N,N)
r3 = r2 * r  # (N,N)

# Mask for i!=j calculations
mask = ~jnp.eye(N, dtype=bool)  # (N,N)

# Compute initial Newtonian accelerations
prefac = 1.0 / r3
prefac = jnp.where(mask, prefac, 0.0)
a_newt = -jnp.sum(prefac[:, :, None] * dx * gms[None, :, None], axis=1)  # (N,3)

# Move to barycentric frame
x_com = jnp.sum(x * gms[:, None], axis=0) / jnp.sum(gms)
v_com = jnp.sum(v * gms[:, None], axis=0) / jnp.sum(gms)
x = x - x_com
v = v - v_com
v2 = jnp.sum(v * v, axis=-1)
vdot = jnp.sum(v[:, None, :] * v[None, :, :], axis=-1)

# Compute constant acceleration terms
# first part of the constant term
a1_arr = jnp.sum((4.0 / C2) * gms / r, axis=1, where=mask)
a1_arr = jnp.broadcast_to(a1_arr, (N, N)).T

a2_arr = jnp.sum((1.0 / C2) * gms / r, axis=1, where=mask)
a2_arr = jnp.broadcast_to(a2_arr, (N, N))

a3_arr = jnp.broadcast_to(-v2 / C2, (N, N)).T

a4_arr = -2.0 * jnp.broadcast_to(v2, (N, N)) / C2

a5_arr = (4.0 / C2) * vdot

a6_0 = jnp.sum(dx * v[None, :, :], axis=-1)
a6_arr = (3.0 / (2 * C2)) * (a6_0**2) / r2

a7_arr = jnp.sum(dx * a_newt[None, :, :], axis=-1) / (2 * C2)

factor1_arr = a1_arr + a2_arr + a3_arr + a4_arr + a5_arr + a6_arr + a7_arr
part1_arr = (
    jnp.broadcast_to(gms, (N, N))[:, :, None]
    * dx
    * factor1_arr[:, :, None]
    / r3[:, :, None]
)


# second part of the constant term
factor2_arr = jnp.sum(dx * (4 * v[:, None, :] - 3 * v[None, :, :]), axis=-1)

part2_arr = (
    jnp.broadcast_to(gms, (N, N))[:, :, None]
    * (
        (
            factor2_arr[:, :, None] * dv / r3[:, :, None]
            + 7 / 2 * a_newt[None, :, :] / r[:, :, None]
        )
    )
    / C2
)

a_const_arr = part1_arr + part2_arr

jnp.sum(a_const_arr, axis=1, where=mask[:, :, None])

Array([[-6.17068615e-04,  9.39287410e-04,  3.20886732e-05],
       [ 4.55402937e-04, -6.94467862e-04, -2.37920049e-05],
       [ 1.46749465e-04,  6.97949623e-05, -3.42430214e-03],
       [-8.10416035e-05,  2.23282535e-05,  3.60617727e-03]],      dtype=float64)

In [198]:
jnp.broadcast_to(v2, (N, N)).T

Array([[0.31974025, 0.31974025, 0.31974025, 0.31974025],
       [0.19135404, 0.19135404, 0.19135404, 0.19135404],
       [0.19135404, 0.19135404, 0.19135404, 0.19135404],
       [0.58445749, 0.58445749, 0.58445749, 0.58445749]], dtype=float64)

In [132]:
jnp.sum(v[0] * v[0])

Array(0.31974025, dtype=float64)

In [14]:
import jax

jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp

import rebound
from jorbit.data.constants import SPEED_OF_LIGHT
from functools import partial


# @partial(jax.jit, static_argnums=(3,))
def gr_full(
    x: jnp.ndarray,  # positions (N,3)
    v: jnp.ndarray,  # velocities (N,3)
    gms: jnp.ndarray,  # gms (N,)
    max_iterations: int = 10,
) -> jnp.ndarray:

    N = x.shape[0]
    C2 = SPEED_OF_LIGHT**2
    C2 = 100.0

    # Calculate pairwise differences
    dx = x[:, None, :] - x[None, :, :]  # (N,N,3)
    r2 = jnp.sum(dx * dx, axis=-1)  # (N,N)
    r = jnp.sqrt(r2)  # (N,N)
    r3 = r2 * r  # (N,N)

    # Mask for i!=j calculations
    mask = ~jnp.eye(N, dtype=bool)  # (N,N)

    # Compute initial Newtonian accelerations
    prefac = 1.0 / r3
    prefac = jnp.where(mask, prefac, 0.0)
    a_newt = -jnp.sum(prefac[:, :, None] * dx * gms[None, :, None], axis=1)  # (N,3)
    print(a_newt.shape)

    # Move to barycentric frame
    x_com = jnp.sum(x * gms[:, None], axis=0) / jnp.sum(gms)
    v_com = jnp.sum(v * gms[:, None], axis=0) / jnp.sum(gms)
    x = x - x_com
    v = v - v_com
    v2 = jnp.sum(v * v, axis=-1)

    # ok tbh, I have pretty much no intuition for how these bits
    # work. I made a pure python version that had all the for loops
    # like the original C code, then played around with vectorized versions
    # of each term until it matched. not sure if I'm proud or horrified
    # by the result, but at least now it's all vectorized
    a1 = jnp.sum(4.0 / C2 * gms[None, :] / r, axis=1, where=mask)
    # a1 = jnp.sum(a1[:, None], axis=1, where=mask)
    # a1 = a1.T

    a2 = jnp.sum(1.0 / C2 * gms[None, :] / r, axis=1, where=mask)
    # a2 = jnp.sum(a2[:, None], axis=0, where=mask)

    a3 = -v2 / C2
    # a3 = jnp.sum(a3[:, None], axis=1, where=mask)
    # a3 = a3.T

    a4 = -2.0 * v2 / C2
    # a4 = jnp.sum(a4[:, None], axis=0, where=mask)

    a5 = (4.0 / C2) * jnp.sum(v[:, None, :] * v[None, :, :], axis=-1)
    # a5 = jnp.sum(a5, axis=0, where=mask)

    v_dot = jnp.sum(v[:, None, :] * dx, axis=2, where=mask[:, :, None])
    a6 = (3 / (2 * C2)) * (v_dot**2 / r2)
    # a6 = jnp.sum(a6, axis=0, where=mask)

    a7 = jnp.sum(dx * a_newt[None, :, :], axis=2) / (2 * C2)
    # a7 = jnp.sum(
    #     jnp.sum(dx * a_newt[None, :, :], axis=2) / (2 * C2),
    #     axis=1,
    #     where=mask
    # )
    # a7 = a7.T

    factor1 = a1 + a2 + a3 + a4 + a5 + a6 + a7
    # print(factor1)

    # a_constx += G * particles[j].m * dxij * factor1 / rij3
    # a_consty += G * particles[j].m * dyij * factor1 / rij3
    # a_constz += G * particles[j].m * dzij * factor1 / rij3

    # factor1 is (N,N) and dx is (N,N,3), and gm is (N,)
    # so we need to sum over the N axis
    a_const = gms[:, None, None] * dx * factor1[:, :, None] / r3[:, :, None]
    a_const = jnp.sum(a_const, axis=(2), where=mask[:, :, None])
    print(a_const)
    a_const = jnp.sum(a_const, axis=0)
    print(a_const)

    # jax.debug.print("{x}", x=a_constx)
    return


sim = rebound.Simulation()
sim.add(m=1.0, x=-1.0, y=1.01, z=-0.05, vx=0.01, vy=0.02, vz=0.4)
sim.add(m=1.3, x=1.0, y=-2.01, z=0.04, vx=-0.02, vy=-0.05, vz=-0.6)
sim.add(m=0.01, x=10.0, y=-2.01, z=0.04, vx=-0.02, vy=-0.05, vz=-0.6)
sim.add(m=0.01, x=10.0, y=-2.01, z=0.4, vx=-0.02, vy=-0.05, vz=0.6)
sim.integrate(1e-300)
# sim.move_to_com()

x0 = jnp.array([[p.x, p.y, p.z] for p in sim.particles])
v0 = jnp.array([[p.vx, p.vy, p.vz] for p in sim.particles])

gr_full(
    x=x0,
    v=v0,
    gms=jnp.array([p.m for p in sim.particles]),
    max_iterations=20,
)

(4, 3)
[[ 0.00000000e+00 -6.75056697e-06  7.35323958e-06 -8.00355621e-05]
 [-8.57580476e-06  0.00000000e+00 -2.56243938e-04  2.76823902e-04]
 [ 1.52429405e-07  1.72169014e-06  0.00000000e+00  1.14487398e-03]
 [ 1.69303636e-06 -8.95779611e-07  2.20626789e-04  0.00000000e+00]]
[-6.73033899e-06 -5.92465644e-06 -2.82639092e-05  1.34166232e-03]


In [9]:
0.023855854677982673 - 0.02385585467798268

-6.938893903907228e-18

In [87]:
def test(x, v, gms):
    N = x.shape[0]
    C2 = SPEED_OF_LIGHT**2
    C2 = 100.0

    # Calculate pairwise differences
    dx = x[:, None, :] - x[None, :, :]  # (N,N,3)
    r2 = jnp.sum(dx * dx, axis=-1)  # (N,N)
    r = jnp.sqrt(r2)  # (N,N)
    r3 = r2 * r  # (N,N)

    # Mask for i!=j calculations
    mask = ~jnp.eye(N, dtype=bool)  # (N,N)

    # Compute initial Newtonian accelerations
    prefac = 1.0 / r3
    prefac = jnp.where(mask, prefac, 0.0)
    a_newt = -jnp.sum(prefac[:, :, None] * dx * gms[None, :, None], axis=1)  # (N,3)

    # Move to barycentric frame
    x_com = jnp.sum(x * gms[:, None], axis=0) / jnp.sum(gms)
    v_com = jnp.sum(v * gms[:, None], axis=0) / jnp.sum(gms)
    x = x - x_com
    v = v - v_com
    v2 = jnp.sum(v * v, axis=-1)

    xs = x[:, 0]
    ys = x[:, 1]
    zs = x[:, 2]
    vxs = v[:, 0]
    vys = v[:, 1]
    vzs = v[:, 2]
    axs = a_newt[:, 0]
    ays = a_newt[:, 1]
    azs = a_newt[:, 2]

    dx = xs[:, jnp.newaxis] - xs[jnp.newaxis, :]
    dy = ys[:, jnp.newaxis] - ys[jnp.newaxis, :]
    dz = zs[:, jnp.newaxis] - zs[jnp.newaxis, :]

    # Compute distance matrices
    r2 = dx**2 + dy**2 + dz**2
    r = jnp.sqrt(r2)
    r3 = r2 * r

    # Create masks to exclude self-interactions
    mask = ~jnp.eye(N, dtype=bool)

    dist_matrix = jnp.sqrt(
        (xs[:, jnp.newaxis] - xs[jnp.newaxis, :]) ** 2
        + (ys[:, jnp.newaxis] - ys[jnp.newaxis, :]) ** 2
        + (zs[:, jnp.newaxis] - zs[jnp.newaxis, :]) ** 2
    )

    # Mask out self-interactions
    dist_matrix = jnp.where(mask, dist_matrix, jnp.inf)

    # Compute a1 terms
    a1_terms = (4.0 / C2) * gms[jnp.newaxis, :] / dist_matrix
    a1 = jnp.sum(a1_terms, axis=1)

    # Compute a2 terms
    a2_terms = (1.0 / C2) * gms[:, jnp.newaxis] / dist_matrix.T
    a2 = jnp.sum(a2_terms, axis=1)

    # Velocity-related terms
    v2 = vxs**2 + vys**2 + vzs**2
    a3 = -v2 / C2
    a4 = -2 * v2 / C2

    # Compute velocity dot products
    v_dot = (
        vxs[:, jnp.newaxis] * vxs[jnp.newaxis, :]
        + vys[:, jnp.newaxis] * vys[jnp.newaxis, :]
        + vzs[:, jnp.newaxis] * vzs[jnp.newaxis, :]
    )
    a5 = (4 / C2) * v_dot

    # Velocity-distance projections
    v_dist_dot = (
        dx * vxs[jnp.newaxis, :] + dy * vys[jnp.newaxis, :] + dz * vzs[jnp.newaxis, :]
    )
    a6 = (3 / (2 * C2)) * (v_dist_dot**2 / r2) * mask

    # Acceleration-distance projections
    a_dist_dot = (
        dx * axs[jnp.newaxis, :] + dy * ays[jnp.newaxis, :] + dz * azs[jnp.newaxis, :]
    ) / (2 * C2)

    # Compute overall factor
    factor1 = (
        a1
        + a2
        + a3
        + a4
        + jnp.sum(a5, axis=1)
        + jnp.sum(a6, axis=1)
        + jnp.sum(a_dist_dot, axis=1)
    )
    print(factor1)

    # Compute constant acceleration components
    a_constx = jnp.sum(gms * dx * factor1[:, jnp.newaxis] / r3, axis=1, where=mask)
    a_consty = jnp.sum(gms * dy * factor1[:, jnp.newaxis] / r3, axis=1, where=mask)
    a_constz = jnp.sum(gms * dz * factor1[:, jnp.newaxis] / r3, axis=1, where=mask)
    return a_constx


test(
    x=x0,
    v=v0,
    gms=jnp.array([p.m for p in sim.particles]),
)

[0.02368016 0.00804794 0.00835965 0.01253183]


Array([-0.00129781,  0.00033639,  0.00019611,  0.0002933 ], dtype=float64)

In [107]:
def dumb(x, v, gms):
    N = x.shape[0]
    C2 = SPEED_OF_LIGHT**2
    C2 = 100.0

    # Calculate pairwise differences
    dx = x[:, None, :] - x[None, :, :]  # (N,N,3)
    r2 = jnp.sum(dx * dx, axis=-1)  # (N,N)
    r = jnp.sqrt(r2)  # (N,N)
    r3 = r2 * r  # (N,N)

    # Mask for i!=j calculations
    mask = ~jnp.eye(N, dtype=bool)  # (N,N)

    # Compute initial Newtonian accelerations
    prefac = 1.0 / r3
    prefac = jnp.where(mask, prefac, 0.0)
    a_newt = -jnp.sum(prefac[:, :, None] * dx * gms[None, :, None], axis=1)  # (N,3)

    # Move to barycentric frame
    x_com = jnp.sum(x * gms[:, None], axis=0) / jnp.sum(gms)
    v_com = jnp.sum(v * gms[:, None], axis=0) / jnp.sum(gms)
    x = x - x_com
    v = v - v_com

    a_const = jnp.zeros((N, 3))
    for i in jnp.arange(N):
        a_constx = 0.0
        a_consty = 0.0
        a_constz = 0.0
        for j in jnp.arange(N):
            print(f"i: {i}, j: {j}")
            if i == j:
                continue
            dxij = x[i, 0] - x[j, 0]
            dyij = x[i, 1] - x[j, 1]
            dzij = x[i, 2] - x[j, 2]
            rij2 = dxij**2 + dyij**2 + dzij**2
            rij = jnp.sqrt(rij2)
            rij3 = rij2 * rij

            # a1 = sum(
            #         (4.0 / C2)
            #         * G
            #         * particles[k].m
            #         / math.sqrt(
            #             (ps_b[i].x - ps_b[k].x) ** 2
            #             + (ps_b[i].y - ps_b[k].y) ** 2
            #             + (ps_b[i].z - ps_b[k].z) ** 2
            #         )
            #         for k in range(N)
            #         if k != i
            #     )
            # _a1 = 4.0 / C2 * gms / rij
            # _a1 = jnp.where(jnp.arange(N) != i, _a1, 0.0)
            # a1 = jnp.sum(_a1)
            print(x.shape)
            print(x[i, 0][jnp.newaxis, :].shape)
            print(
                jnp.sqrt(
                    (x[i, 0][None, :] - x[:, 0]) ** 2
                    + (x[i, 1][None, :] - x[:, 1]) ** 2
                    + (x[i, 2][None, :] - x[:, 2]) ** 2
                ).shape
            )
    #         _a1 = (4.0 / C2) * gms[:,None]  / jnp.sqrt(
    #                     (x[i,0][:,None] - x[:,0]) ** 2
    #                     + (x[i,1][:,None] - x[:,1]) ** 2
    #                     + (x[i,2][:,None] - x[:,2]) ** 2
    #                 )
    #         print(_a1.shape)
    #         print(f"a1: {a1}")

    #         a_constx += gms[j] * dxij * (a1 + a2 + a3 + a4 + a5 + a6 + a7) / rij3
    #         a_consty += gms[j] * dyij * (a1 + a2 + a3 + a4 + a5 + a6 + a7) / rij3
    #         a_constz += gms[j] * dzij * (a1 + a2 + a3 + a4 + a5 + a6 + a7) / rij3
    #     a_const = a_const.at[i, 0].set(a_constx)
    #     a_const = a_const.at[i, 1].set(a_consty)
    #     a_const = a_const.at[i, 2].set(a_constz)

    # return a_const[0]


dumb(
    x=x0,
    v=v0,
    gms=jnp.array([p.m for p in sim.particles]),
)

i: 0, j: 0
i: 0, j: 1
(4, 3)


IndexError: Too many indices: 0-dimensional array indexed with 1 regular index.

In [3]:
import jax

jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp

from jplephem.spk import SPK
import astropy.units as u
from astropy.time import Time
from astropy.utils.data import download_file

In [1]:
import jax

jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp

import rebound
from jorbit.accelerations.gr import gr_full

sim = rebound.Simulation()
sim.add(m=1.0, x=-1.0, y=1.01, z=-0.05, vx=0.01, vy=0.02, vz=0.4)
sim.add(m=1.3, x=1.0, y=-2.01, z=0.04, vx=-0.02, vy=-0.05, vz=-0.6)
sim.integrate(1e-300)
# sim.move_to_com()

x0 = jnp.array([[p.x, p.y, p.z] for p in sim.particles])
v0 = jnp.array([[p.vx, p.vy, p.vz] for p in sim.particles])

tmp = gr_full(
    x=x0,
    v=v0,
    gms=jnp.array([sim.particles[0].m, sim.particles[1].m]),
    max_iterations=20,
)[0][1]

gr_res = tmp
for i in range(3):
    print(float(tmp[i]))

a1: 0.014351450235174385
a2: 0.0035878625587935963
a3: -0.0032132362948960307
a4: -0.0064264725897920615
a5: 0.012852945179584123
a6: nan
a7: 0.0
factor 1: 0.01472607649907195
part 1: -0.00013212442775999202
part 2: -0.0005261776269904688
a_const: -0.0006583020547504608
-0.041635035251997805
0.06286385744278797
-0.001917279759932883


In [8]:
0.002759894275995074 - 0.0035878625587935963

-0.0008279682827985221

In [1]:
import jax

jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp
from jorbit.accelerations.tmp_weird import rebx_calculate_gr_full
import rebound

sim = rebound.Simulation()
sim.add(m=1.0, x=-1.0, y=1.01, z=-0.05, vx=0.01, vy=0.02, vz=0.4)
sim.add(m=1.3, x=1.0, y=-2.01, z=0.04, vx=-0.02, vy=-0.05, vz=-0.6)
sim.integrate(1e-300)
particles = list(sim.particles)

p1, p2 = rebx_calculate_gr_full(
    particles, C2=10**2, G=1.0, max_iterations=10, gravity_ignore_10=False
)
py_res = jnp.array([p2.ax, p2.ay, p2.az])
print("***")
print(p2.ax)
print(p2.ay)
print(p2.az)

mass: 1.0
a1: 0.014351450235174387
a2: 0.002759894275995074
a3: -0.0032132362948960307
a4: -0.0038026465028355378
a5: -0.0098868809073724
a6: 8.142489755004931e-07
a7: 0.001379947137997537
factor1: 0.0015893421930385297
part 1: -8.686965529213308e-05
part 2: -0.0005261776269904688
a1: 0.011039577103980297
a2: 0.0035878625587935967
a3: -0.0019013232514177689
a4: -0.0064264725897920615
a5: -0.0098868809073724
a6: 1.3760807685958317e-06
a7: 0.0017939312793967981
factor1: -0.0017919297256429423
part 1: -7.534046681322893e-05
part 2: 0.0005266423008151755
a_const: -0.0006130472822826019
***
-0.04159971869967043
0.06281052944877362
-0.001915690515078151


In [1]:
import jax

jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp

from jorbit.accelerations.tmp_weird import test2
import rebound

sim = rebound.Simulation()
sim.add(m=1.0, x=-1.0, y=1.01, z=-0.05, vx=0.01, vy=0.02, vz=0.4)
sim.add(m=1.3, x=1.0, y=-2.01, z=0.04, vx=-0.02, vy=-0.05, vz=-0.6)
sim.integrate(1e-300)
particles = list(sim.particles)

test2(particles, C2=10**2, G=1.0, max_iterations=10, gravity_ignore_10=False)

a5: 0.012852945179584123
a5: -0.0098868809073724
a5: -0.0098868809073724
a5: 0.0076052930056710755
a_const: -0.0004621980407230713


Array([[-4.62198041e-04,  7.04816743e-04,  3.89442748e-05],
       [ 3.00452592e-04, -4.58795220e-04, -3.07546028e-05]],      dtype=float64)

In [10]:
import jax

jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp
import rebound
import reboundx

sim = rebound.Simulation()
sim.add(m=1.0, x=-1.0, y=1.01, z=-0.05, vx=0.01, vy=0.02, vz=0.4)
sim.add(m=1.3, x=1.0, y=-2.01, z=0.04, vx=-0.02, vy=-0.05, vz=-0.6)
sim.integrate(1e-300)
a0 = jnp.array([[p.ax, p.ay, p.az] for p in sim.particles])
sim = rebound.Simulation()
sim.add(m=1.0, x=-1.0, y=1.01, z=-0.05, vx=0.01, vy=0.02, vz=0.4)
sim.add(m=1.3, x=1.0, y=-2.01, z=0.04, vx=-0.02, vy=-0.05, vz=-0.6)
rebx = reboundx.Extras(sim)
gr = rebx.load_force("gr_full")
gr.params["c"] = 10
gr.params["max_iterations"] = 10
rebx.add_force(gr)
sim.integrate(1e-300)
a1 = sim.particles[0].ax
print(sim.particles[1].ax)
print(sim.particles[1].ay)
print(sim.particles[1].az)
a1 = jnp.array([[p.ax, p.ay, p.az] for p in sim.particles])
reb_res = jnp.array([sim.particles[1].ax, sim.particles[1].ay, sim.particles[1].az])

-0.04159971869967043
0.06281052944877362
-0.001915690515078151


In [11]:
py_res - reb_res

Array([0., 0., 0.], dtype=float64)

In [16]:
gr_res - py_res

Array([-3.53165523e-05,  5.33279940e-05, -1.58924485e-06], dtype=float64)

In [ ]:
import rebound
import reboundx

sim = rebound.Simulation()
sim.add(m=1.0, x=-10.0, y=1.01, z=-0.05, vx=0.01, vy=0.02, vz=0.4)
sim.add(m=1.3, x=10.0, y=-1.01, z=0.05, vx=-0.01, vy=-0.02, vz=-0.4)
sim.move_to_com()
rebx = reboundx.Extras(sim)
gr = rebx.load_force("gr_full")
gr.params["c"] = 10
gr.params["max_iterations"] = 10
rebx.add_force(gr)
sim.integrate(1e-300)
a1 = sim.particles[0].ax
print(sim.particles[1].ax)
print(sim.particles[1].ay)
print(sim.particles[1].az)

1.0

In [ ]:
sim = rebound.Simulation()
# sim.add(m=1.0, x=-1.0, y=0.01, z=-0.01)
# sim.add(m=1.0, x=1.0, y=-0.01, z=0.01)
sim.add(m=1.0, x=-1.0, y=1.01, z=-0.05, vx=0.01, vy=0.02, vz=0.4)
sim.add(m=1.3, x=1.0, y=-1.01, z=0.05, vx=-0.01, vy=-0.02, vz=-0.4)
sim.integrate(1e-300)
particles = list(sim.particles)

p1, p2 = rebx_calculate_gr_full(
    particles, C2=10**2, G=1.0, max_iterations=10, gravity_ignore_10=False
)
print("***")
print(p2.ax)
print(p2.ay)
print(p2.az)

mass: 1.0
a_const: -0.0028343167716862105
***
-0.08475123395303748
0.08560221079401008
-0.004191579361234195


In [23]:
-0.004191579361234195 - -0.004191579361234195

0.0

In [19]:
import rebound
import reboundx

sim = rebound.Simulation()
sim.add(m=1.0, x=-1.0, y=1.01, z=-0.05, vx=0.01, vy=0.02, vz=0.4)
sim.add(m=1.3, x=1.0, y=-1.01, z=0.05, vx=-0.01, vy=-0.02, vz=-0.4)
sim.integrate(1e-300)
a0 = jnp.array([[p.ax, p.ay, p.az] for p in sim.particles])
sim = rebound.Simulation()
sim.add(m=1.0, x=-1.0, y=1.01, z=-0.05, vx=0.01, vy=0.02, vz=0.4)
sim.add(m=1.3, x=1.0, y=-1.01, z=0.05, vx=-0.01, vy=-0.02, vz=-0.4)
rebx = reboundx.Extras(sim)
gr = rebx.load_force("gr_full")
gr.params["c"] = 10
gr.params["max_iterations"] = 10
rebx.add_force(gr)
sim.integrate(1e-300)
a1 = sim.particles[0].ax
print(sim.particles[1].ax)
print(sim.particles[1].ay)
print(sim.particles[1].az)
a1 = jnp.array([[p.ax, p.ay, p.az] for p in sim.particles])
a1 - a0

-0.08475123395303748
0.08560221079401008
-0.004191579361234195


Array([[-0.00279483,  0.00281808, -0.00020209],
       [ 0.00215982, -0.00217795,  0.00015397]], dtype=float64)